# Orchestrateur

Ce notebook gère l'exécution des benchmarks pour tous les agents.

**Processus :**
1.  Définit la liste de tous les agents à tester.
2.  Pour chaque agent, il appelle votre script `RunAgents.py` en utilisant `subprocess`.
3.  Il capture la sortie (stdout) et les erreurs (stderr) de chaque script.
4.  Il mesure le temps total d'exécution pour chaque agent.

**Important :** Vous devez avoir un script nommé `RunAgents.py` dans le même dossier que ce notebook pour que cela fonctionne.

Une fois ce notebook terminé, vous pouvez ouvrir `Report.ipynb` pour analyser les résultats.

## 1. Configuration des Agents

Liste des noms de classe des agents (doit correspondre aux noms de fichiers `.py` dans le dossier `agents/`).

In [7]:
agent_files_to_run = [
    'BestAgentMLP',
    'ExtraTreeAgent5features',
    'ExtraTreeAgent9features',
    'KNNFaiss',
    'LGBEXTHybride',
    'LGBMAgent5features',
    'MLPAgentBase',
    'MLPBoost10features',
    'MLPV2_1'
    
]
"""
Voici une liste plus large d'autres agents que vous pouvez lancer : 





"""
print(f"{len(agent_files_to_run)} agents sont configurés pour le test.")

9 agents sont configurés pour le test.


## 2. Boucle d'Exécution

Cette cellule exécute le script `RunAgents.py` pour chaque agent de la liste. Contrairement aux tests qu'on a utilisé pour selectionner nos agents, ici on affichera uniquement la première tache pour un soucis de lisibilité et de temps.

Les scripts des agents sont donnés dans le dossier `agents` si vous souhaitez les essayer. 

In [8]:
import subprocess
import time
import sys
import os

print("DÉBUT DU BENCHMARK GLOBAL\n" + "="*70)
overall_start_time = time.time()
total_agents = len(agent_files_to_run)

for i, agent_name in enumerate(agent_files_to_run):
    print(f"\n--- [Agent {i+1}/{total_agents}] Lancement de : {agent_name} ---")
    agent_start_time = time.time()
    
    # Utilise 'sys.executable' pour s'assurer d'utiliser le même interpréteur python
    # AJOUT DE -X utf8 pour forcer l'encodage
    command = [sys.executable, '-X', 'utf8', 'RunAgents.py', agent_name]
    
    # On récupère l'environnement actuel et on force l'encodage pour python
    env = os.environ.copy()
    env['PYTHONIOENCODING'] = 'utf-8'

    try:
        # Exécute le script en tant que sous-processus
        # On capture en bytes (capture_output=True) puis on décode manuellement
        result = subprocess.run(command, 
                                capture_output=True, 
                                env=env, 
                                timeout=600 # Ajout d'un timeout de 10min au cas où
                               )
        
        # Décodage manuel en remplaçant les erreurs
        stdout_str = result.stdout.decode('utf-8', errors='replace')
        stderr_str = result.stderr.decode('utf-8', errors='replace')
        
    except Exception as e:
        print(f"!!! ERREUR SYSTÈME (subprocess) pour {agent_name} : {e} !!!")
        print(f"--- [Agent {i+1}/{total_agents}] {agent_name} ÉCHOUÉ (Erreur Subprocess) ---")
        continue 

    agent_elapsed_time = time.time() - agent_start_time
    
    # --- Afficher les résultats ---
    print("--- STDOUT  ---")
    print(stdout_str)
    
    if result.returncode == 0:
        # Succès
        if stderr_str: # Afficher les warningv s'il y'en a 
            print("--- STDERR  ---")
            print(stderr_str)
        print(f"--- [Agent {i+1}/{total_agents}] {agent_name} TERMINÉ en {agent_elapsed_time:.2f}s ---")
    else:
        print(f"!!! ERREUR D'EXÉCUTION pour {agent_name} (Code de retour: {result.returncode}) !!!")
        print("--- STDERR  ---")
        print(stderr_str)
        print(f"--- [Agent {i+1}/{total_agents}] {agent_name} ÉCHOUÉ en {agent_elapsed_time:.2f}s ---")

overall_elapsed_time = time.time() - overall_start_time
print("\n" + "="*70)
print(f"BENCHMARK GLOBAL TERMINÉ.")
print(f"Temps total d'exécution : {overall_elapsed_time:.2f} secondes")
print("Vous pouvez maintenant ouvrir 'Report.ipynb' pour analyser les résultats.")

DÉBUT DU BENCHMARK GLOBAL

--- [Agent 1/9] Lancement de : BestAgentMLP ---
--- STDOUT  ---
Mode 1: Exécution d'un seul agent : BestAgentMLP

ÉVALUATION DE L'AGENT : BestAgentMLP

--- Tâche 1/7 ---
Tâche 1: Précision = 98.56%, Temps = 54.0587s, CPU = 87.4062s
--- Tâche 2/7 ---
Tâche 2: Précision = 98.67%, Temps = 53.5235s, CPU = 87.5938s
--- Tâche 3/7 ---
Tâche 3: Précision = 98.49%, Temps = 52.7704s, CPU = 85.9062s
--- Tâche 4/7 ---
Tâche 4: Précision = 98.62%, Temps = 51.4188s, CPU = 84.2344s
--- Tâche 5/7 ---
Tâche 5: Précision = 98.62%, Temps = 53.7507s, CPU = 86.8438s
--- Tâche 6/7 ---
Tâche 6: Précision = 98.57%, Temps = 52.7397s, CPU = 84.7500s
--- Tâche 7/7 ---
Tâche 7: Précision = 98.56%, Temps = 52.8016s, CPU = 84.7188s

Résultats sauvegardés dans : soumissions\BestAgentMLP_results.json

Résumé pour BestAgentMLP:
  Précision moyenne: 98.58% ± 0.05%
  Temps total (réel): 371.06s
  Temps total (CPU) : 601.45s
  Pic de RAM          : 1016.64 MB

--- [Agent 1/9] BestAgentMLP TERMI